# Library import and Spark initialization

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
import pandas as pd
import matplotlib.pyplot as plt
import cml.data_v1 as cmldata
import csv, os

# Sample in-code customization of spark configurations
#from pyspark import SparkContext
#SparkContext.setSystemProperty('spark.executor.cores', '1')
#SparkContext.setSystemProperty('spark.executor.memory', '2g')

CONNECTION_NAME = "pdnd-prod-dl-1"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

print("SparkSession created.")

# SQL query definition

In [ ]:
sql_query = """
    select 
    s.id_stazione as stazione,
    sp.creditor.idPA as EC,
    count(*) as TRX
    from pagopa.silver_positive sp
    inner join pagopa.bronze_cfg_pa pa on pa.id_dominio = sp.creditor.idPA
    inner join pagopa.bronze_cfg_pa_stazione_pa psp on psp.fk_pa = pa.obj_id
    inner join pagopa.bronze_cfg_stazioni s on s.obj_id = psp.fk_stazione
    where 1=1
    and sp.paymentinfo.paymentdatetime >= CAST('2025-07-01 00:00:00' AS TIMESTAMP)
    and sp.paymentinfo.paymentdatetime <  CAST('2025-08-01 00:00:00' AS TIMESTAMP)
    and sp.creditor.idStation = '15376371009_51'
    and sp.debtorposition.iuv like concat(CAST(psp.segregazione AS STRING), '%')
    group by 
    s.id_stazione,
    sp.creditor.idPA
    order by TRX desc
    """

# Query execution and result preview

In [ ]:
# Spark query ececution
print("Spark query execution...")
results_df = spark.sql(sql_query)

# Dataframe caching
results_df.cache()

# Generate csv file

In [ ]:
local_dir = "/home/cdsw/report"
os.makedirs(local_dir, exist_ok=True)
local_csv = os.path.join(local_dir, "report_stazioni_trx.csv")

# write header from schema, then stream rows
with open(local_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    cols = [f.name for f in results_df.schema.fields]
    writer.writerow(cols)
    for r in results_df.toLocalIterator():   # streams; no big collect
        writer.writerow([r[c] for c in cols])

print(f"[OK] CSV written on driver: {local_csv}")

# Computing the 10 stations with the most transactions...

In [ ]:
print("Calcolo delle 10 stazioni con più transazioni...")

# Aggregate by station, sum transactions, get top 10
top_10_station_df = results_df.groupBy("stazione") \
    .agg(sum("TRX").alias("total_trx")) \
    .orderBy(col("total_trx").desc()) \
    .limit(10)

print("DataFrame ready for visualizzation:")
top_10_station_df.show()

# Plotting libraries like Matplotlib work with local (in-memory) data, not distributed DataFrames. 
## We then need to convert our small top_10_station_df DataFrame (which only contains 10 rows) into a Pandas DataFrame using .toPandas().

In [ ]:
print("Convert from Spark DF into Pandas DF for plotting...")
top_10_pandas_df = top_10_station_df.toPandas()

# Bar chart creation
print("Bar chart creation...")
plt.style.use('ggplot') # Stile del grafico
fig, ax = plt.subplots(figsize=(12, 8)) # Dimensioni del grafico

ax.bar(top_10_pandas_df['stazione'], top_10_pandas_df['total_trx'], color='skyblue')

# Add label and title
ax.set_xlabel('ID Stazione', fontsize=12)
ax.set_ylabel('Numero Totale di Transazioni', fontsize=12)
ax.set_title('Top 10 Stazioni per Numero di Transazioni (Luglio 2025)', fontsize=16)

# Rotate the labels
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Show graph
plt.show()